In [ ]:
# https://github.com/salesforce/LAVIS/blob/main/lavis/models/blip2_models/blip2_qformer.py#L90
image = samples["image"]
text = samples["text_input"]

image是图像的原始数据，shape是[batch_size, num_channels, height, width]。
text是原始文本。

In [ ]:
image_embeds = self.ln_vision(self.visual_encoder(image))
image_atts = torch.ones(image_embeds.size()[:-1], dtype=torch.long).to(image.device)

visual_encoder就是Figure 2中的Image Encoder，比如EVA。image_embeds的shape是[batch_size, num_patches, embed_dim]
image_atts将图片的attention全部设置为1，在之后的三个损失函数的计算中，图片特征都是可以全部互相注意的。

In [ ]:
query_tokens = self.query_tokens.expand(image_embeds.shape[0], -1, -1)

query_tokens是示意图中的learned queries，扩张成image_embeds一样的batch size。

In [ ]:
query_output = self.Qformer.bert(
    query_embeds=query_tokens,
    encoder_hidden_states=image_embeds,
    encoder_attention_mask=image_atts,
    use_cache=True,
    return_dict=True,
)

这部分的代码细节就开始多起来了。这里的bert是Q-Former的核心参数，其实Q-Former就是基于bert实现的。下文提到的bert，都是指self.Qformer.bert。

bert的block层包含self attention层和cross attention层。它既可以接收query_embeds，也可以接收input_ids。如果只有query_embeds，就不需要过embedding层了。

这里的encoder_hidden_states和encoder_attention_mask，和原始Transformer中的encoder不是一个东西。在Q-Former中，看到encoder_hidden_states和encoder_attention_mask，就说明有cross attention发生。

详细的代码解释，见附录。

In [ ]:
image_feats = F.normalize(
    self.vision_proj(query_output.last_hidden_state), dim=-1
)

query_output.last_hidden_state是图片和queries计算完cross attention的隐状态。
vision_proj是维度映射。
image_feats是图片的最终特征。

In [ ]:
text_tokens = self.tokenizer(
    text,
    padding="max_length",
    truncation=True,
    max_length=self.max_txt_len,
    return_tensors="pt",
).to(image.device)

tokenizer将原始文字转化为inpud_ids.

In [ ]:
text_output = self.Qformer.bert(
    text_tokens.input_ids,
    attention_mask=text_tokens.attention_mask,
    return_dict=True,
)

bert是多功能的，在这里又可以编码文本。这就是原文所说的share the same self-attention layers。
attention_mask和input_ids同形状，pad部分设为0，其他是1。也就是说，文本之前也是全部可以互相注意。

In [ ]:
text_feat = F.normalize(
    self.text_proj(text_output.last_hidden_state[:, 0, :]), dim=-1
)

同理，text_feat是最终的文本特征。

In [ ]:
image_feats_all = concat_all_gather(image_feats)  # [batch_size*num_gpu, num_query_tokens, embed_dim]
text_feat_all = concat_all_gather(text_feat)  # [batch_size*num_gpu, embed_dim]

将多个gpu的数据合并。

In [ ]:
sim_q2t = torch.matmul(
    image_feats.unsqueeze(1), text_feat_all.unsqueeze(-1)
).squeeze()
# [batch_size, batch_size*num_gpu, num_query_tokens]

image_feats 的原始形状是 [batch_size, num_query_tokens, embed_dim]。unsqueeze(1) 在第1维度上增加一个维度，结果形状变为 [batch_size, 1, num_query_tokens, embed_dim]。

text_feat_all 的原始形状是 [batch_size*num_gpu, embed_dim]。unsqueeze(-1) 在最后一个维度上增加一个维度，结果形状变为 [batch_size*num_gpu, embed_dim, 1]。

torch.matmul将两个张量相乘。假设 A 的形状为 [a, b, c, d]，B 的形状为 [e, f, g]，如果 d 与 e 匹配，结果张量的形状为 [a, b, c, g]。
在此上下文中：
image_feats.unsqueeze(1) 的形状是 [batch_size, 1, num_query_tokens, embed_dim]
text_feat_all.unsqueeze(-1) 的形状是 [batch_size*num_gpu, embed_dim, 1]
通过矩阵乘法 torch.matmul，embed_dim 是匹配的维度，结果形状为 [batch_size, batch_size*num_gpu, num_query_tokens, 1]

最终结果 sim_q2t 表示每个图像查询 token 与所有文本特征的相似度矩阵，其中：
- 第一维度 batch_size 对应当前批次中的图像。
- 第二维度 batch_size*num_gpu 对应所有收集到的文本特征（跨多个 GPU）。
- 第三维度 num_query_tokens 对应每个图像的查询 token。

In [ ]:
# image-text similarity: aggregate across all query tokens
sim_i2t, _ = sim_q2t.max(-1)
sim_i2t = sim_i2t / self.temp

计算最大值，得到[batch_size, batch_size*num_gpu]

In [ ]:
# text-query similarity: [batch_size, batch_size*num_gpu, num_query_tokens]
sim_t2q = torch.matmul(
    text_feat.unsqueeze(1).unsqueeze(1), image_feats_all.permute(0, 2, 1)
).squeeze()

# text-image similarity: aggregate across all query tokens
sim_t2i, _ = sim_t2q.max(-1)
sim_t2i = sim_t2i / self.temp  # [batch_size, batch_size*num_gpu]

同理。

In [ ]:
rank = dist.get_rank()
bs = image.size(0)
targets = torch.linspace(rank * bs, rank * bs + bs - 1, bs, dtype=int).to(image.device)

torch.linspace(start, end, steps, dtype) 生成一个从 start 到 end 的等差数列，共有 steps 个元素。
这里生成的是目标标签，具体解释如下：
- start 是 rank * bs，即当前进程（或 GPU）的排名乘以批次大小。这是当前批次的起始索引。
- end 是 rank * bs + bs - 1，即当前批次的结束索引。
- steps 是 bs，即生成 bs 个目标标签。

假设有4个 GPU，每个 GPU 的批次大小为 32。
- 对于 rank=0 的 GPU，生成的目标标签为 [0, 1, 2, ..., 31]。
- 对于 rank=1 的 GPU，生成的目标标签为 [32, 33, 34, ..., 63]。
- 对于 rank=2 的 GPU，生成的目标标签为 [64, 65, 66, ..., 95]。
- 对于 rank=3 的 GPU，生成的目标标签为 [96, 97, 98, ..., 127]。

In [ ]:
if "image_id" in samples.keys(): #coco retrieval finetuning
    image_ids = samples["image_id"].view(-1,1)
    image_ids_all = concat_all_gather(image_ids)
    pos_idx = torch.eq(image_ids, image_ids_all.t()).float()       
    sim_targets = pos_idx / pos_idx.sum(1,keepdim=True)   
    sim_targets = 0.9 * sim_targets + 0.1 * torch.ones_like(sim_targets) / sim_targets.size(1)

    loss_t2i = -torch.sum(F.log_softmax(sim_t2i, dim=1)*sim_targets,dim=1).mean()
    loss_i2t = -torch.sum(F.log_softmax(sim_i2t, dim=1)*sim_targets,dim=1).mean()     
    loss_itc = (loss_t2i+loss_i2t)/2  
else:                     
    loss_itc = (
        F.cross_entropy(sim_i2t, targets, label_smoothing=0.1)
        + F.cross_entropy(sim_t2i, targets, label_smoothing=0.1)
    ) / 2

解释else部分。
sim_i2t 是图像到文本的相似度矩阵，形状为 [batch_size, batch_size*num_gpu]，表示当前批次中的每个图像与所有文本特征的相似度。
sim_t2i 是文本到图像的相似度矩阵，形状为 [batch_size, batch_size*num_gpu]，表示当前批次中的每个文本与所有图像特征的相似度。
交叉熵计算的是负对数，targets告诉损失函数每个图像（或文本）对应的正确文本（或图像）的索引。最小化交叉熵，就相当于最大化图片和其对应文本的相似度。